# Data Wrangling with MongoDB

## Open Street Map: Pittsburgh, Pennsylvania, USA

This Project will go through the process of assessing and cleaning OpenStreetMap data for the Pittsburgh, PA area: https://mapzen.com/data/metro-extracts/metro/pittsburgh_pennsylvania/ </br> 
After exploring and performing audting on sample of the area, I have discovered problems that compromises the quality of the data. 

I will show the process of audting, followed by problems encountered, then my attemps at cleaning and correcting some of these problems, finally will write the data to JSON file and import it to MongoDB and run some queries. 

## 1. Exploring and Auditing the Data:  

Due to the large number of tags and variables in this dataset, I will set my focus of the address data of nodes. 

### Problems: 

1. One of the main concerns is the street names having different forms and abbreviation, for example: 
<br>
<b>Airport Boulevard 
<br> 
Airport Blvd</b>

2. Address Zip code, this field generated many problems regarding the format, for example some have xxxxx-xxxx, the extended four digits are used for postal delivery services. Also, some have leading character representing the state, this will also be removed.

<b> Streets: </b>
First, after running and exploring the auditing of street types multible times, I have constructed list of all expected street types,  I also have a list of the correct street types, meaning they don't require any cleaning. For the ones that don't fall into the correct street types, there will be method to correct it based on special mapping. 
<br> 
One on the problems with street names here is that they may have suffix so this must be taken into considiration before implying the street type is incorrect. 

Running this method: audit_street_names(filename): 
<br>
It will give and overview understanding of the process on sample data and some statistic info of the addr:street variable. 

<b> ZIP-Codes: </b> 
In order to clean this field I will remove the extended four digits part, in addition to all leading characters.
<br>
Running this method: audit_postal_codes(filename): 
<br>
For example: 15213-1503 => 15213


In [18]:
# import packages: 

import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict  # available in Python 2.5 and newer
import re
import codecs
import json
from pymongo import MongoClient
import os
import math

In [4]:
OSM_FILE = "pittsburgh_pennsylvania.osm"
SAMPLE_FILE = "sample_sample_pittsburgh_pennsylvania.osm"
DB_NAME = "pittsburgh_pennsylvania"
JSON_FILE = "pittsburgh_pennsylvania.osm.json"
COLLECTION_NAME = "pittsburgh_pennsylvania_data"

In [67]:
# method to update street names that need correction based on supplied mappaing: 
def update_street_name (mapping, street_name, suffix):
    name = street_name
    street_type = street_type_re.search(street_name).group() 
    if  street_type not in correct_street_types:
        street_data['incorrect_streets'] += 1
        name = street_name.replace(street_type, mapping[street_type])
        street_data['corrected_streets'] += 1
    if suffix in street_suffix:
        name = name + " - " + suffix
        street_data['corrected_streets_suffix'] += 1
    return name

In [68]:
zip_code_re = re.compile(r'\d+', re.IGNORECASE) 

def update_postal_code (postcode):
    m = zip_code_re.search(postcode)
    if m:
        match = m.group(0)
        return match

### 2. Writing to JSON file:  

Now that the data we're interetsed in is ready for processing, this phase will iterate over OpenStreetMap data and convert it after cleaning to JSON file, without modifying the original data file. I have chose JSON becuase in the next phase it will be imported to MongoDB. 

In [172]:
if __name__ == '__main__':
    data = write_json(OSM_FILE, True)

### 3. Proccessing data with MongoDB

After exporting the cleaned data to JSON file, it's imported to MongoDB using this command: 

``` 
mongoimport -d pittsburgh_pennsylvania -c pittsburgh_pennsylvania_data --file pittsburgh_pennsylvania.osm.json 
```

Now will do some data proccessing and querying to provide an overview statistics about the data, and other ideas. 

In [1]:
# method to get database by its name: 
def get_db(db_name):
    # For local use
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

In [8]:
# info about the data: 
if __name__ == "__main__":
    db = get_db(DB_NAME)
    print "Database Name: " + db.name
    
    print "Sample Data Point: "
    pprint.pprint (db.pittsburgh_pennsylvania_data.find_one())
    
    print "Data Statisisc: "
    pprint.pprint(get_overview_statistics(OSM_FILE, JSON_FILE, db))

Database Name: pittsburgh_pennsylvania
Sample Data Point: 
{u'_id': ObjectId('58f9354e6aa9fc9fc8e23f32'),
 u'address': {},
 u'created': {u'changeset': u'134337',
              u'timestamp': u'2007-07-09T03:10:11Z',
              u'uid': u'867',
              u'user': u'tscofield',
              u'version': u'1'},
 u'id': u'31479671',
 u'pos': [u'-80.10257', u'40.1688251'],
 u'type': u'node',
 u'visible': u'true'}
Data Statisisc: 
{'JSON file size - MB': 726.0528079999999,
 'OSM file size - MB': 476.13142,
 'Total documents count': 2370018,
 'Total nodes count': 2146449,
 'Total unique users': 1424,
 'Total ways count': 223569}


In [6]:
def get_overview_statistics(osm_filename, json_filename, db):
    statistics = {}
    statistics['OSM file size - MB'] = os.path.getsize(osm_filename) * (1e-6)
    statistics['JSON file size - MB'] = os.path.getsize(json_filename) * (1e-6)
    statistics['Total documents count'] = db.pittsburgh_pennsylvania_data.find().count()
    statistics['Total nodes count'] = db.pittsburgh_pennsylvania_data.find({"type": "node"}).count()
    statistics['Total ways count'] = db.pittsburgh_pennsylvania_data.find({"type": "way"}).count()
    statistics['Total unique users'] = len(db.pittsburgh_pennsylvania_data.distinct("created.uid"))
    
    return statistics

In [76]:
def query_db (db):    
    print "Top Contributing User: "
    top_users = db.pittsburgh_pennsylvania_data.aggregate([{"$group":{"_id":"$created.uid", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":1}])
    for user in top_users:
        print user

    print "Top Type of Amenities: "
    top_amenities = db.pittsburgh_pennsylvania_data.aggregate([{"$match":{"amenity":{"$exists":1}}}, {"$group":{"_id":"$amenity","count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":1}])
    for amenity in top_amenities:
        pprint.pprint(amenity)
        
    print "Top Type of Shops: "     
    top_shops = db.pittsburgh_pennsylvania_data.aggregate([{"$match":{"shop":{"$exists":1}}}, {"$group":{"_id":"$shop","count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":1}])
    for shop in top_shops:
        pprint.pprint(shop)
     
    print "Top Type of cuisines: "
    top_cuisines = db.pittsburgh_pennsylvania_data.aggregate([{"$match":{"cuisine":{"$exists":1}}}, {"$group":{"_id":"$cuisine","count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":1}])
    for cuisine in top_cuisines:
        pprint.pprint(cuisine)
    
    print "Most Popular Shops: "
    shop_results = db.pittsburgh_pennsylvania_data.aggregate([{"$match":{"shop":{"$exists":1}, "name":{"$exists":1}}}, {"$group":{"_id":"$name","count":{"$sum":1}}}, {"$sort":{"count":-1}},{"$limit":3}])
    for result in shop_results:
        pprint.pprint(result)
            

In [77]:
# get additional info about db: 
if __name__ == "__main__":
    db = get_db(DB_NAME)
    query_db(db)

Top Contributing User: 
{u'count': 307248, u'_id': u'2835510'}
Top Type of Amenities: 
{u'_id': u'parking', u'count': 4440}
Top Type of Shops: 
{u'_id': u'car_repair', u'count': 284}
Top Type of cuisines: 
{u'_id': u'american', u'count': 174}
Most Popular Shops: 
{u'_id': u'Giant Eagle', u'count': 54}
{u'_id': u'Dollar General', u'count': 26}
{u'_id': u'Sheetz', u'count': 20}


### 3. Other Ideas About  the dataset: 

One idea that concerns me is how schools serves districts, and how we can improve and utilize their services, for example I will attempt to get the number of houses whithin 10 miles: I will get any school using mongoDB query, finding the surrounding premises will require some advanced geometry calculation so for the sake of this report I will look for houses on the same street as the school, to give an overview of the idea. Of course this is not very accurate because not all of these points would be households but given the metadata we have we can’t distinguish between houses or other types of locations. To further improve such analysis we need more data about the school and district households.  
<br>
I would like to see integrated data about public transportations, for example bus routes, and some detailed data about rides, hours, dates, whether. 




In [80]:
    print "Education Stats: "
    edu_results = db.pittsburgh_pennsylvania_data.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity": {'$in': ["university","school", "college", "grade_school", "childcare", "education_centre"]}}}, {"$group":{"_id":"$amenity","count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])
    for result in edu_results:
        pprint.pprint(result)
        
        
    school_sample = db.pittsburgh_pennsylvania_data.find_one({"amenity": "school"})
    print "Sample School: "
    pprint.pprint (school_sample)
    nearby_houses = db.pittsburgh_pennsylvania_data.find ({"address.street": school_sample["address"]["street"]}, {"type": "node"}).count()
    print "Nearby houses count: "
    print nearby_houses


Education Stats: 
{u'_id': u'school', u'count': 1401}
{u'_id': u'university', u'count': 66}
{u'_id': u'college', u'count': 14}
{u'_id': u'childcare', u'count': 3}
{u'_id': u'education_centre', u'count': 1}
{u'_id': u'grade_school', u'count': 1}
Sample School: 
{u'_id': ObjectId('58faf8feaf2d3979fe905765'),
 u'address': {u'city': u'Penn Hills',
              u'housenumber': u'3123',
              u'postcode': u'15147',
              u'state': u'PA',
              u'street': u'Long Hollow Road'},
 u'amenity': u'school',
 u'created': {u'changeset': u'19066334',
              u'timestamp': u'2013-11-23T03:40:50Z',
              u'uid': u'1813632',
              u'user': u'JeanLiu',
              u'version': u'5'},
 u'id': u'158554556',
 u'name': u'Sunset Valley',
 u'pos': [u'-79.7434092', u'40.3055684'],
 u'type': u'node',
 u'visible': u'true'}
Nearby houses count: 
7250


### 4. Conclusion and Imporvements: 

#### Imporvements: 
Now that we have part of open street map data cleaned, it is important to mention some improvements that could be done to further enhance the quality of data. One major component it the source of data, some nodes have data imported from TIGER (http://wiki.openstreetmap.org/wiki/TIGER), while doing some research it's advised that it may have incorrect data points and requires fix-up. Nonetheless, it can be seen that some data only apparent in TIGER tags, we can still use these but more cleaning and validating the accuracy and consistency is highly required. 
<br>
I have had several problems with the data up to this point after wrangling and cleaning. The accuracy and correctness is debatable! The address and position data when doing quick google map search are not consistent, there are many duplications as well but it requires more digging and familiarity with the area to detect. 
<br>
<br>
I think there could be some suggestion to improve the data collection process, for example add some soft of rating of the data, where users can rate each others entries. 
<br>
There could be some implemented analysis to establish baseline for the metadata required for each type of nodes, for example if the entry is restaurant the user would be required to enter more details; cuisine, hours, phone number. 
<br>
This analysis could also beneficial to detect main details of certain location based on nearby data points, for example: state name, country, etc, and provide it for the user to ensure accuracy. Also for big areas it would be useful to ask users to provide as specific details as possible, for example: district name, neighborhood, etc.   
#### Anticipated Problems:
When dealing with data there’s always chance of errors, so we can't depend on suggested analysis to ensure the availability of further details, this could also cause limitation of data collection. 
Also, when asking users to rate entries as suggested to improve quality, there could be problems in getting users to do so, we need to implement mechanisms or reward system to motivate users. 
<br>
#### Conclusion:
In conclusion the improvements are countless but nonetheless the data collected is huge and with more wrangling and cleaning we can establish much more insights. 
